In [1]:
import os
import pandas as pd
import numpy as np
import pickle

from sklearn.utils.class_weight import compute_class_weight
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import *

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
plt.style.use('seaborn')

Using TensorFlow backend.


---

### First, copy-paste the setup from the notebook that produced the pretrained classweights.
This time we load also the test-data into memory
(Note to self: learn how to make imports from other notebooks...)

In [2]:
global_params = pickle.load(open("global_params.p", "rb"))
unique_words = global_params['unique_words']
num_words = global_params['num_words']
padded_length = global_params['padded_length']
n_classes = global_params['n_classes']
class_weights = global_params['class_weights']

global_params

{'unique_words': 277303,
 'num_words': 277304,
 'padded_length': 679,
 'n_classes': 6,
 'class_weights': array([ 1.26825655,  0.72736371,  0.27602776, 13.23801959, 30.29201502,
         9.49559404])}

In [9]:
X_train = np.load(os.path.join("data", "Kdata", "X_train.npy"))
y_train = np.load(os.path.join("data", "Kdata", "y_train.npy"))
X_val = np.load(os.path.join("data", "Kdata", "X_val.npy"))
y_val = np.load(os.path.join("data", "Kdata", "y_val.npy"))
X_test = np.load(os.path.join("data", "Kdata", "X_test.npy"))
y_test = np.load(os.path.join("data", "Kdata", "y_test.npy"))

In [5]:
from keras import backend as K

# Custom metrics.
# Note that any metric is computed per batch during training
# (hence one needs large batches for it to make sense).
# Also, there is the implicit 0.5 treshold in the K.round function below

def macroPrec(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    positives = K.sum(K.round(y_pred), axis=0)
    precision = true_positives / (positives + K.epsilon())

    macroPrec = K.mean( precision )
    
    return macroPrec


def macroRecall(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    actual_ones = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    recall = true_positives / (actual_ones + K.epsilon())

    macroRecall = K.mean( recall )
    
    return macroRecall


def macroF1(y_true, y_pred):
    
    def recall(y_true, y_pred):

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
        actual_ones = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
        recall = true_positives / (actual_ones + K.epsilon())
        return recall

    
    def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
        positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
        precision = true_positives / (positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    macroF1 = K.mean( 2*((precision*recall)/(precision+recall+K.epsilon())) )
    
    return macroF1



# Custom loss functions
# No clippping or rounding as those are not differentiable
# Note to self: these functions are already taking into account class imbalances, so remeber
# not to set the class_weight argument in fitting method when using them as the loss 

# semi macroF1
def fuzzy_macroF1_flip(y_true, y_pred):
    
    def recall(y_true, y_pred):

        true_positives = K.sum(y_true * y_pred, axis=0)
        actual_ones = K.sum(y_true, axis=0)
        recall = true_positives / (actual_ones + K.epsilon())
        return recall

    
    def precision(y_true, y_pred):
        
        true_positives = K.sum(y_true * y_pred, axis=0)
        positives = K.sum(y_pred, axis=0)
        precision = true_positives / (positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    macroF1 = K.mean( 2*((precision*recall)/(precision+recall+K.epsilon())) ) 
    
    return 1-macroF1


# weighted categorical crossentropy
def my_crossE(y_true, y_pred):
    
    class_entropy = K.sum(y_true * K.log(y_pred+K.epsilon()), axis=0)
    weighted_entropy = class_weights * class_entropy
    return -K.mean(weighted_entropy)/4000  # arbitrary coefficient to make it close to [0-1] interval


# take average of the last two, because why not
def my_loss(y_true, y_pred):
    return 0.5*(fuzzy_macroF1_flip(y_true, y_pred) + my_crossE(y_true, y_pred))

In [7]:
# some other global learning settings

batch_size = 3000
patience = 2
optimizer = 'adam' # 'rmsprop'
my_metrics = [macroPrec, macroF1, macroRecall, my_crossE, "categorical_accuracy"]

In [8]:
# auxiliary functions
    

# to plot the learning history,
# i.e. loss and metrics on each train_batch and validation_batch
def plot_history(history):
    
    plt.figure(figsize=(12, 5))
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(loss) + 1)
    
    plt.subplot(1, 2, 1)    
    plt.plot(x, loss, 'ob', label='Training loss')
    plt.plot(x, val_loss, '-b', label='Validation loss')
    plt.title('Training and val losses') 
    plt.legend()
    
    all_metrics = np.array(list(set(history.history.keys()) - set(['loss', 'val_loss', 'lr'])))
    metrics = all_metrics[[not metric.startswith('val_') for metric in all_metrics]]
    plt.subplot(1, 2, 2)
    markers = ['b', 'r',  'g', 'y', 'm', 'c', 'k']
    marker = zip(markers, markers)
    for metric in metrics:
        results = history.history[metric]
        val_results = history.history['val_'+metric]
        mark1, mark2 = next(marker)
        plt.plot(x, results, 'o'+mark1, label=metric)
        plt.plot(x, val_results, '-'+mark2, label='Validation '+metric)
        plt.title('Training and val metrics')
    plt.legend()
    
#     plt.subplot(1, 3, 3)
#     lr = history.history['lr'] # learning rate
#     plt.plot(x, lr, 'k', label='Learning rate')
#     plt.title('Learning rate')  
#     plt.legend()
    

# choose callback functions to be given in model.fit(...)
def callbacks(LRfactor=0.5):
    return [ReduceLROnPlateau(factor=LRfactor, patience = patience//2),
            EarlyStopping(monitor='val_macroF1', mode='max', patience=patience)
           ]

---

In [47]:
# the workhorse

class BlackBox():
    
    def __init__(self, layers, loss, metrics, callbacks=None, class_weight=class_weights, epochs=10,\
                 embedd = None, embedd_dims=[None]*3, train_weights=False):
        
        self.layers = layers
        self.loss = loss
        self.metrics = metrics
        self.callbacks = callbacks
        self.class_weight = class_weight
        self.epochs = epochs
        self.train_weigts = train_weights
        self.history = None
        self.test_results = None
        
        self.model = Sequential()
        if embedd:
            self.input_dim = embedd_dims[0] # num_words
            self.output_dim = embedd_dims[1] # dimension of the embedding space
            self.input_length = embedd_dims[2] # padded_length
            with open(embedd, "rb") as f:
                self.embedd_weights = pickle.load(f)
            
            self.model.add(Embedding(input_dim=self.input_dim,\
                                     output_dim=self.output_dim,\
                                     input_length=self.input_length,\
                                     weights=self.embedd_weights,\
                                     trainable=self.train_weigts))
            self.model.layers[0].trainable = self.train_weigts  # szczeżonego panbug szczeże
            
        for layer in layers:
            self.model.add(layer)
            
        self.model.add(Dense(n_classes, activation='softmax'))
        self.model.compile(loss=self.loss, optimizer=optimizer, metrics=self.metrics)
    
    def summary(self):
        return self.model.summary()
    
    def fit(self, epochs=50):
        self.history = self.model.fit(
            X_train, y_train,
            class_weight=self.class_weight,
            epochs=self.epochs,
            batch_size=batch_size,
            #validation_split=0.1,
            validation_data=(X_val, y_val),
            callbacks=self.callbacks,
            verbose = 1        
            )
        
    def reset(self):
        
        self.history = None
        self.test_results = None
        self.model.compile(loss=self.loss, optimizer=optimizer, metrics=self.metrics)
        
    def evaluate(self, X_test, y_test):
        print(f'Loss function: {self.loss.__name__ if callable(self.loss) else self.loss}. Metrics: {[metric.__name__ if callable(metric) else metric for metric in self.metrics]}')
        print("Predicting for X_test, comparing with y_test")
        self.test_results = self.model.evaluate(X_test, y_test)
        return self.test_results
    
    def plot(self):
        plot_history(self.history)
        
    def save_embedd(self, name):
        file = os.path.join("my_embeddings", f"weights_{name}.p")
        with open(file, 'wb') as f:
            pickle.dump(self.model.layers[0].get_weights(), f)
        
    def Ksave(self, name):
        file = os.path.join("my_embeddings", f"Kmodel_from_{name}.h5")
        self.model.save(file)
    
    def save_hist(self, name):
        file = os.path.join("my_embeddings", f"history_of_{name}.p")
        with open(file, 'w') as f:
            json.dump(self.history.history, f)



---
---

# Loss = *categorical_crossentropy*

---

### Dense nets

In [50]:
# embedding: keras layer, trained
# structure: dense layers

layers1a = [Dense(50, activation='relu'),
           MaxPooling1D(4, 2),
           Dropout(0.1),
           Dense(50, activation='relu'),
            MaxPooling1D(4, 2),
           Dropout(0.1),
            Flatten()
          ]

model1a = BlackBox(layers=layers1a, loss='categorical_crossentropy', metrics=my_metrics,\
                   epochs=40, callbacks = callbacks(),\
                   embedd = "embedd_weights_50dim_GlblAvg_F1flip_10epochs.p",\
                   embedd_dims=[num_words, 50, padded_length])
model1a.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, 679, 50)           13865200  
_________________________________________________________________
dense_70 (Dense)             (None, 679, 50)           2550      
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 338, 50)           0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 338, 50)           0         
_________________________________________________________________
dense_71 (Dense)             (None, 338, 50)           2550      
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 168, 50)           0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 168, 50)           0         
__________

In [51]:
model1a.fit()

Train on 500000 samples, validate on 200000 samples
Epoch 1/50
500000/500000 [==============================] - 635s 1ms/step - loss: 0.2856 - macroPrec: 0.7784 - macroF1: 0.6681 - macroRecall: 0.6264 - my_crossE: 1.5867 - val_loss: 0.2150 - val_macroPrec: 0.8239 - val_macroF1: 0.7790 - val_macroRecall: 0.7464 - val_my_crossE: 1.3139
Epoch 2/50
500000/500000 [==============================] - 625s 1ms/step - loss: 0.1924 - macroPrec: 0.8805 - macroF1: 0.8248 - macroRecall: 0.7857 - my_crossE: 1.2923 - val_loss: 0.2122 - val_macroPrec: 0.8262 - val_macroF1: 0.7836 - val_macroRecall: 0.7522 - val_my_crossE: 1.3136
Epoch 3/50
500000/500000 [==============================] - 638s 1ms/step - loss: 0.1875 - macroPrec: 0.8888 - macroF1: 0.8383 - macroRecall: 0.8018 - my_crossE: 1.2795 - val_loss: 0.2102 - val_macroPrec: 0.8230 - val_macroF1: 0.7855 - val_macroRecall: 0.7586 - val_my_crossE: 1.3001
Epoch 4/50
500000/500000 [==============================] - 641s 1ms/step - loss: 0.1852 - macro

KeyboardInterrupt: 

---

### Convolutions

In [27]:
# embedding: keras layer, trained
# structure: convolution layers

layers2a = [Conv1D(40, 4),
            MaxPooling1D(4, 2),
            Dropout(0.1),
            Conv1D(40, 4),
            MaxPooling1D(4, 2),
            Dropout(0.1),
            Flatten()
           ]

model2a = BlackBox(layers=layers2a, loss='categorical_crossentropy', metrics=my_metrics,\
                  embedd = "my_embedd_weights_50dim_GlblAvg.p",\
                  embedd_dims=[num_words, 50, padded_length])
model2a.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 679, 50)           13865200  
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 676, 40)           8040      
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 337, 40)           0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 337, 40)           0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 334, 40)           6440      
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 166, 40)           0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 166, 40)           0         
__________

In [ ]:
model2a.fit()

---

### Recurrent NNEts

In [42]:
# embedding: keras layer, trained
# structure: birectional fully connected RNN layer + dense

layers3a = [Bidirectional(SimpleRNN(130, dropout=0.1, recurrent_dropout=0.1)),
            Dense(30, activation='relu')
           ]

model3a = BlackBox(layers=layers3a, loss='categorical_crossentropy', metrics=my_metrics,\
                  embedd = "my_embedd_weights_50dim_GlblAvg.p",\
                  embedd_dims=[num_words, 50, padded_length])
model3a.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 679, 50)           13865200  
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 260)               47060     
_________________________________________________________________
dense_62 (Dense)             (None, 30)                7830      
_________________________________________________________________
dense_63 (Dense)             (None, 6)                 186       
Total params: 13,920,276
Trainable params: 55,076
Non-trainable params: 13,865,200
_________________________________________________________________


---

### Short-Long-Term-Memory

In [38]:
# embedding: keras layer, trained
# structure: birectional fully connected RNN layer + dense

layers4a = [Bidirectional(LSTM(60, dropout=0.1, recurrent_dropout=0.1)),
            Dense(20, activation='relu')
           ]

model4a = BlackBox(layers=layers4a, loss='categorical_crossentropy', metrics=my_metrics,\
                  embedd = "my_embedd_weights_50dim_GlblAvg.p",\
                  embedd_dims=[num_words, 50, padded_length])
model4a.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 679, 50)           13865200  
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 120)               53280     
_________________________________________________________________
dense_54 (Dense)             (None, 20)                2420      
_________________________________________________________________
dense_55 (Dense)             (None, 6)                 126       
Total params: 13,921,026
Trainable params: 55,826
Non-trainable params: 13,865,200
_________________________________________________________________


---
---

# Loss = fuzzy_macroF1_flip

---

### Dense nets

---

---

---